In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from fpdf import FPDF
from datetime import datetime

# Define the folder path containing the CSV files
folder_path = '/Users/ajfoeckler/Downloads/dockhounds_games'
save_folder_path = '/Users/ajfoeckler/Downloads/dockhounds_pitching_data'

# Create the save directory if it doesn't exist
os.makedirs(save_folder_path, exist_ok=True)

# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Load all CSV files and concatenate them into a single DataFrame
data_frames = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    data_frame = pd.read_csv(file_path)
    data_frames.append(data_frame)

original_data = pd.concat(data_frames, ignore_index=True)

# Ensure 'Date' column is in datetime format
original_data['Date'] = pd.to_datetime(original_data['Date'], errors='coerce')

# Filter out rows with NaT in the 'Date' column
original_data = original_data.dropna(subset=['Date'])

# Filter data for Lake Country pitchers
lake_country_pitchers = original_data[
    (original_data['PitcherTeam'] == 'LAK_COU10') &
    (original_data['Date'].dt.year == 2025)
].copy()

pitch_type_map = {
    'Four-Seam': 'Four-Seam',
    'Cutter': 'Cutter',
    'Sinker': 'Sinker',
    'Slider': 'Slider',
    'Curveball': 'Curveball',
    'Changeup': 'Changeup',
    'Splitter': 'Splitter',
    'Sweeper': 'Sweeper',
    'Slurve':  'Slurve',
}    
lake_country_pitchers['AutoPitchType'] = (
    lake_country_pitchers['AutoPitchType']
      .map(pitch_type_map)
      .fillna(lake_country_pitchers['AutoPitchType'])
)    

# Get a list of unique pitchers
unique_pitchers = lake_country_pitchers['Pitcher'].unique()

# Define the strike zone dimensions
strike_zone_top = 3.673333
strike_zone_bottom = 1.524166667
strike_zone_left = -0.83083333
strike_zone_right = 0.83083333

def is_in_strike_zone(row):
    return strike_zone_bottom <= row['PlateLocHeight'] <= strike_zone_top and strike_zone_left <= row['PlateLocSide'] <= strike_zone_right

def save_plot_as_image(filename):
    plt.savefig(filename, bbox_inches='tight')
    plt.close()

def plot_balls_in_zone(snider_data, pitcher_name):
    balls_in_zone = snider_data[(snider_data['InStrikeZone']) & (snider_data['PitchCall'] == 'BallCalled')]
    
    colors = {
        'Four-Seam': '#001f3f',   # Navy
        'Slider': '#377EB8',      # Blue
        'Curveball': '#4DAF4A',   # Green
        'Sinker': '#FF7F00',      # Orange
        'Changeup': '#7FDBFF',    # Columbia Blue
        'Cutter': '#984EA3',       # Purple
        'Sweeper':   '#FF1493',  
        'Slurve':    '#2E8B57'
    }

    plt.figure(figsize=(10, 8))
    plt.plot([strike_zone_left, strike_zone_right], [strike_zone_bottom, strike_zone_bottom], color='black', linewidth=2)
    plt.plot([strike_zone_left, strike_zone_right], [strike_zone_top, strike_zone_top], color='black', linewidth=2)
    plt.plot([strike_zone_left, strike_zone_left], [strike_zone_bottom, strike_zone_top], color='black', linewidth=2)
    plt.plot([strike_zone_right, strike_zone_right], [strike_zone_bottom, strike_zone_top], color='black', linewidth=2)
    for pitch_type in balls_in_zone['AutoPitchType'].unique():
        subset = balls_in_zone[balls_in_zone['AutoPitchType'] == pitch_type]
        plt.scatter(subset['PlateLocSide'], subset['PlateLocHeight'], label=pitch_type, color=colors.get(pitch_type, 'gray'), s=100, edgecolors='w', linewidth=0.5)
    plt.xlim(-2.5, 2.5)
    plt.ylim(0, 5)
    plt.xlabel('Horizontal Location (feet)')
    plt.ylabel('Vertical Location (feet)')
    plt.title('Pitches in Zone Called Balls - ' + pitcher_name)
    plt.legend(loc='upper right', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    save_plot_as_image('balls_in_zone.png')

def generate_pdf_for_pitcher(pitcher_name, snider_data, game_date, include_balls_in_zone):
    snider_data = snider_data.copy()
    snider_data['InStrikeZone'] = snider_data.apply(is_in_strike_zone, axis=1)

    swing_conditions = ['FoulBallNotFieldable', 'FoulBallFieldable', 'InPlay', 'StrikeSwinging']
    whiff_conditions = ['StrikeSwinging']

    zone_percentage = snider_data.groupby('AutoPitchType')['InStrikeZone'].mean() * 100
    strikes = snider_data[~snider_data['PitchCall'].isin(['BallCalled'])]
    strike_percentage = strikes.groupby('AutoPitchType').size() / snider_data.groupby('AutoPitchType').size() * 100

    swing_percentage = snider_data.groupby('AutoPitchType')['PitchCall'].apply(lambda x: (x.isin(swing_conditions).mean()) * 100)
    whiff_percentage = snider_data.groupby('AutoPitchType')['PitchCall'].apply(lambda x: (x.isin(whiff_conditions).mean()) / x.isin(swing_conditions).mean() * 100)
    chase_percentage = snider_data[snider_data['InStrikeZone'] == False].groupby('AutoPitchType')['PitchCall'].apply(lambda x: (x.isin(swing_conditions).mean()) * 100)

    zero_zero_data = snider_data[(snider_data['Balls'] == 0) & (snider_data['Strikes'] == 0)]
    zero_zero_strike_percentage = zero_zero_data.groupby('AutoPitchType')['PitchCall'].apply(lambda x: (~x.isin(['BallCalled'])).mean() * 100)

    one_one_data = snider_data[(snider_data['Balls'] == 1) & (snider_data['Strikes'] == 1)]
    one_one_strike_percentage = one_one_data.groupby('AutoPitchType')['PitchCall'].apply(lambda x: (~x.isin(['BallCalled'])).mean() * 100)

    pitch_counts = snider_data['AutoPitchType'].value_counts()
    total_pitches = pitch_counts.sum()
    usage_percentage = (pitch_counts / total_pitches * 100).round(1)

    pitch_outcomes = pd.DataFrame({
        '#': pitch_counts,
        'Usage%': usage_percentage,
        'Swing%': swing_percentage,
        'Whiff%': whiff_percentage,
        'Zone%': zone_percentage,
        'Chase%': chase_percentage,
        '0-0 Str%': zero_zero_strike_percentage,
        '1-1 Str%': one_one_strike_percentage,
        'Strike%': strike_percentage
    }).reset_index().rename(columns={'index': 'Pitch Type'}).sort_values(by='Usage%', ascending=False).round(2)

    snider_pitch_characteristics = snider_data.groupby('AutoPitchType').agg({
        'RelSpeed': ['mean', 'max'],
        'HorzBreak': 'mean',
        'InducedVertBreak': 'mean',
        'SpinRate': ['mean', 'max'],
        'RelSide': 'mean',
        'RelHeight': 'mean',
        'Extension': 'mean',
        'VertApprAngle': 'mean',
        'HorzApprAngle': 'mean'
    }).reset_index()

    snider_pitch_characteristics.columns = [
        'Pitch Type', 'Velocity', 'Max Velo', 'HB', 'IVB', 'Spin Rate', 'Max Spin',
        'R-Side', 'R-Height', 'Ext', 'VAA', 'HAA'
    ]

    snider_pitch_characteristics = snider_pitch_characteristics.merge(pitch_outcomes[['Pitch Type', 'Usage%']], on='Pitch Type')
    snider_pitch_characteristics = snider_pitch_characteristics.sort_values(by='Usage%', ascending=False).round(2)

    colors = {
        'Four-Seam': '#001f3f',   # Navy
        'Slider': '#377EB8',      # Blue
        'Curveball': '#4DAF4A',   # Green
        'Sinker': '#FF7F00',      # Orange
        'Changeup': '#7FDBFF',    # Columbia Blue
        'Cutter': '#984EA3',       # Purple
        'Sweeper':   '#FF1493',
        'Slurve':    '#2E8B57',
    }

    pitch_types = snider_data['AutoPitchType'].unique()
    rows = (len(pitch_types) + 1) // 2

    plt.figure(figsize=(14, rows * 5))
    for idx, pitch in enumerate(pitch_types, start=1):
        plt.subplot(rows, 2, idx)
        subset = snider_data[snider_data['AutoPitchType'] == pitch]
        plt.plot([strike_zone_left, strike_zone_right], [strike_zone_bottom, strike_zone_bottom], color='black', linewidth=2)
        plt.plot([strike_zone_left, strike_zone_right], [strike_zone_top, strike_zone_top], color='black', linewidth=2)
        plt.plot([strike_zone_left, strike_zone_left], [strike_zone_bottom, strike_zone_top], color='black', linewidth=2)
        plt.plot([strike_zone_right, strike_zone_right], [strike_zone_bottom, strike_zone_top], color='black', linewidth=2)
        plt.scatter(subset['PlateLocSide'], subset['PlateLocHeight'], label=pitch, color=colors.get(pitch, 'gray'), s=50, edgecolors='w', linewidth=0.5)
        plt.axvline(subset['PlateLocSide'].mean(), color='blue', linestyle='--', linewidth=1, label='Avg Horz Location')
        plt.axhline(subset['PlateLocHeight'].mean(), color='green', linestyle='--', linewidth=1, label='Avg Vert Location')
        plt.xlim(-2.5, 2.5)
        plt.ylim(0, 5)
        plt.xlabel('Horizontal Location (feet)')
        plt.ylabel('Vertical Location (feet)')
        plt.title(f'{pitch} Locations in Strike Zone - {pitcher_name}')
        plt.legend(loc='upper right', fontsize=8)
        plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    save_plot_as_image('strike_zone.png')

    plt.figure(figsize=(10, 8))
    for pitch in snider_pitch_characteristics['Pitch Type']:
        subset = snider_data[snider_data['AutoPitchType'] == pitch]
        plt.scatter(subset['PitchNo'], subset['RelSpeed'], label=pitch, color=colors.get(pitch, 'gray'), s=50, edgecolors='w', linewidth=0.5)
        plt.axhline(subset['RelSpeed'].mean(), color=colors.get(pitch, 'gray'), linestyle='--', linewidth=1)
    plt.xlabel('Pitch #')
    plt.ylabel('Velocity (mph)')
    plt.title('Pitch Velocity Trend - ' + pitcher_name)
    plt.legend(loc='upper right', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    save_plot_as_image('velocity_trend.png')

    plt.figure(figsize=(10, 8))
    for pitch in snider_pitch_characteristics['Pitch Type']:
        subset = snider_data[snider_data['AutoPitchType'] == pitch]
        plt.scatter(subset['HorzBreak'], subset['InducedVertBreak'], label=pitch, color=colors.get(pitch, 'gray'), s=100, edgecolors='w', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.axhline(0, color='black', linewidth=0.5)
    plt.xlim(-30, 30)
    plt.ylim(-30, 30)
    plt.xlabel('Horz Break')
    plt.ylabel('Ind. Vert Break')
    plt.title('Pitch Movement (Pitcher POV) - ' + pitcher_name)
    plt.legend(loc='upper right', fontsize=10)
    plt.grid(True, linestyle='-', alpha=0.7)
    save_plot_as_image('pitch_movement.png')

    plt.figure(figsize=(10, 8))
    for pitch in snider_pitch_characteristics['Pitch Type']:
        subset = snider_data[snider_data['AutoPitchType'] == pitch]
        plt.scatter(subset['RelSide'], subset['RelHeight'], label=pitch, color=colors.get(pitch, 'gray'), s=100, edgecolors='w', linewidth=0.5)
    plt.axvline(snider_data['RelSide'].mean(), color='blue', linestyle='--', linewidth=1, label='Avg Horz Release')
    plt.axhline(snider_data['RelHeight'].mean(), color='green', linestyle='--', linewidth=1, label='Avg Vert Release')
    plt.xlim(-5.0, 5.0)
    plt.ylim(4.0, 8.0)
    plt.xlabel('Horizontal Release (feet)')
    plt.ylabel('Vertical Release (feet)')
    plt.title('Release Point (Pitcher POV) - ' + pitcher_name)
    plt.legend(loc='upper right', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    save_plot_as_image('release_point.png')

    plt.figure(figsize=(10, 8))
    for pitch in snider_pitch_characteristics['Pitch Type']:
        subset = snider_data[snider_data['AutoPitchType'] == pitch]
        plt.scatter(subset['Extension'], subset['RelHeight'], label=pitch, color=colors.get(pitch, 'gray'), s=100, edgecolors='w', linewidth=0.5)
    plt.axvline(snider_data['Extension'].mean(), color='blue', linestyle='--', linewidth=1, label='Avg Extension')
    plt.axhline(snider_data['RelHeight'].mean(), color='green', linestyle='--', linewidth=1, label='Avg Vert Release Height')
    plt.xlim(0.0, 10.0)
    plt.ylim(4.0, 8.0)
    plt.xlabel('Extension (feet)')
    plt.ylabel('Vertical Release Height (feet)')
    plt.title('Extension & Vertical Release Height - ' + pitcher_name)
    plt.legend(loc='upper right', fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.7)
    save_plot_as_image('extension_release.png')

    if include_balls_in_zone:
        plot_balls_in_zone(snider_data, pitcher_name)

    class PDF(FPDF):
        def header(self):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, 'Pitch Analysis Report - ' + pitcher_name, 0, 1, 'C')
        
        def chapter_title(self, title):
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, title, 0, 1, 'L')
            self.ln(10)

        def chapter_body(self, body):
            self.set_font('Arial', '', 12)
            self.multi_cell(0, 10, body)
            self.ln()

        def table(self, dataframe):
            self.set_font('Arial', 'B', 10)
            col_width = self.w / (len(dataframe.columns) + 1)
            row_height = self.font_size * 1.5
            
            for col_name in dataframe.columns:
                self.cell(col_width, row_height, str(col_name), border=1)
            self.ln(row_height)
            
            self.set_font('Arial', '', 10)
            for row in dataframe.itertuples(index=False):
                for item in row:
                    self.cell(col_width, row_height, str(item), border=1)
                self.ln(row_height)

    pdf = PDF(orientation='L', unit='mm', format='A4')
    pdf.add_page()

    pdf.chapter_title('Pitch Outcomes and Characteristics Tables')

    pdf.cell(0, 10, 'Pitch Outcomes', 0, 1, 'L')
    pdf.table(pitch_outcomes)

    pdf.ln(10)
    pdf.cell(0, 10, 'Pitch Characteristics', 0, 1, 'L')
    pdf.table(snider_pitch_characteristics)

    pdf.add_page(orientation='P')
    pdf.chapter_title('Strike Zone with Pitch Locations')
    pdf.image('strike_zone.png', x=10, y=40, w=pdf.w - 20)

    pdf.add_page(orientation='P')
    pdf.chapter_title('Pitch Velocity Trend')
    pdf.image('velocity_trend.png', x=10, y=40, w=pdf.w - 20)

    pdf.add_page(orientation='P')
    pdf.chapter_title('Pitch Movement (Pitcher POV)')
    pdf.image('pitch_movement.png', x=10, y=40, w=pdf.w - 20)

    pdf.add_page(orientation='P')
    pdf.chapter_title('Release Point (Horizontal & Vertical Release)')
    pdf.image('release_point.png', x=10, y=40, w=pdf.w - 20)

    pdf.add_page(orientation='P')
    pdf.chapter_title('Extension & Vertical Release Height')
    pdf.image('extension_release.png', x=10, y=40, w=pdf.w - 20)

    if include_balls_in_zone:
        pdf.add_page(orientation='P')
        pdf.chapter_title('Pitches in Zone Called Balls')
        pdf.image('balls_in_zone.png', x=10, y=40, w=pdf.w - 20)

    pdf_filename = f"{pitcher_name.replace(', ', '_')}_{game_date.strftime('%Y-%m-%d')}.pdf"
    game_folder_path = os.path.join(save_folder_path, game_date.strftime('%Y-%m-%d'))
    os.makedirs(game_folder_path, exist_ok=True)
    pdf_output_path = os.path.join(game_folder_path, pdf_filename)

    pdf.output(pdf_output_path)

    png_files = [
        'strike_zone.png', 'velocity_trend.png', 'pitch_movement.png',
        'release_point.png', 'extension_release.png', 'balls_in_zone.png'
    ]

    for png_file in png_files:
        if os.path.exists(png_file):
            os.remove(png_file)

    print(f"PDF generated successfully for {pitcher_name} on {game_date.strftime('%Y-%m-%d')} and saved to {pdf_output_path}!")

include_balls_in_zone = (
    input("Include 'balls called in zone' graph in the report? (yes/no): ")
    .strip()
    .lower() == 'yes'
)

game_dates = lake_country_pitchers['Date'].unique()

for raw_date in game_dates:
    # convert once per date and make its folder
    game_date = pd.to_datetime(raw_date)
    folder = os.path.join(
        save_folder_path,
        game_date.strftime('%Y-%m-%d')
    )
    os.makedirs(folder, exist_ok=True)

    # One pass per pitcher for this date
    for pitcher_name in unique_pitchers:
        mask = (
            (lake_country_pitchers['Pitcher'] == pitcher_name) &
            (lake_country_pitchers['Date'] == raw_date)
        )
        pitcher_game_data = lake_country_pitchers.loc[mask]
        
        if pitcher_game_data.empty:
            continue

   # Removed Cantleberry split‐by‐inning block; just generate one PDF per pitcher
        generate_pdf_for_pitcher(
            pitcher_name,
            pitcher_game_data,
            game_date,
            include_balls_in_zone
        )

Include 'balls called in zone' graph in the report? (yes/no): yes


/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:256: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:257: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Pitch Analysis Report - ' + pitcher_name, 0, 1, 'C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:260: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:261: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, title,

PDF generated successfully for Nix, Jacob on 2025-05-22 and saved to /Users/ajfoeckler/Downloads/dockhounds_pitching_data/2025-05-22/Nix_Jacob_2025-05-22.pdf!


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:256: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:257: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Pitch Analysis Report - ' + pitcher_name, 0, 1, 'C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:260: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.

PDF generated successfully for Torres, Eric on 2025-05-22 and saved to /Users/ajfoeckler/Downloads/dockhounds_pitching_data/2025-05-22/Torres_Eric_2025-05-22.pdf!


...

PDF generated successfully for Riley, Trey on 2025-07-27 and saved to /Users/ajfoeckler/Downloads/dockhounds_pitching_data/2025-07-27/Riley_Trey_2025-07-27.pdf!


/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:256: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:257: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Pitch Analysis Report - ' + pitcher_name, 0, 1, 'C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:260: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:261: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, title,

PDF generated successfully for Dillard, Trey on 2025-07-27 and saved to /Users/ajfoeckler/Downloads/dockhounds_pitching_data/2025-07-27/Dillard_Trey_2025-07-27.pdf!


/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:114: RuntimeWarning: invalid value encountered in double_scalars
  whiff_percentage = snider_data.groupby('AutoPitchType')['PitchCall'].apply(lambda x: (x.isin(whiff_conditions).mean()) / x.isin(swing_conditions).mean() * 100)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:256: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 12)
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:257: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'Pitch Analysis Report - ' + pitcher_name, 0, 1, 'C')
/var/folders/1j/1qrr3dyd7zd02ny12l57zltw0000gn/T/ipykernel_2103/1637581437.py:260: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated 

PDF generated successfully for Cancellieri, Dominic on 2025-07-27 and saved to /Users/ajfoeckler/Downloads/dockhounds_pitching_data/2025-07-27/Cancellieri_Dominic_2025-07-27.pdf!
